In [ ]:
#####
# MIS 740 - GROUP 3 - FINAL PROJECT
#
# Authors: Jason Jasper, Shanthan Pochampally, Kumar Rishav, Luke Stevens, Curtis Wa
# Date: 11-18-2022
#####

# Imports here, time needed for timestamps
import time
import pandas as pd
import matplotlib.pyplot as plt
# Regex
import re

# CONSTANTS
# Menu Items
QUIT_PROGRAM = 'Q'
ADHOC_PROPERTY_SEARCH = 1
IMPORT_PROPERTY_LIST = 2
IMPORT_DATA_SET = 3
RUN_PROPERTY_SEARCH = 4
GENERATE_REPORT = 5
SUMMARY_REPORT_ONLY = 6
CREATE_PLOTS = 7

# Range of menu items, for comparisons
MENU_RANGE = (1, 2, 3, 4, 5, 6, 7)

# Search types used in property_search function
ADDRESS_SEARCH = 1
PARCEL_SEARCH = 2
ADHOC_SEARCH = 1
MULTI_SEARCH = 2

# Secured property tax rates in San Francisco from 2008 2019
# Source: https://sfcontroller.org/sites/default/files/Documents/Budget/PropertyTaxRevenueAllocation_web_FY2021-22.pdf
PROPERTY_TAX_YEARLY_RATES = {2007: .01141, 2008: .01163, 2009: .01159, 2010: .0164, 2011: .011718, 2012: .011691, 2013: .01188,
2014: .011743, 2015: .011826, 2016: .011792, 2017: .011723, 2018: .011630, 2019: .011801}

# For validating that the dataset imported meets the  minimum requirements for a data search and tax information
TAX_DATASET_MINIMUM_DATA = {'closed_roll_year', 'property_location', 'parcel_number', 'assessed_fixtures_value',
'assessed_improvement_value', 'assessed_land_value', 'assessed_personal_property_value'}

PROPERTY_LIST_MINIMUM_DATA = {'closed_roll_year', 'property_location', 'parcel_number'}

# This regular expression is required to correct the formatting of addresses that are in the dataset. Otherwise the address format
# is not in a very readable format. Addresses that do not match this format usualy do not have a situs (tax address) assigned and are vacant lots.
ADDRESS_REGEX = '^\d{4}\s(\d{4}\s\S*\s?\S*)\s+([A-Z]{2})'

# Dataset dtypes dictionary, pandas generates warnings without this.
# NOTE: THIS ENDED UP MAKING READ_CSV TAKE TWICE AS LONG TO PROCESS THE DATA SET, AND I HAD TO COERCE NA'S TO INT64
# WE MIGHT NOT USE THIS BUT I AM LEAVING IT IN HERE, JUST IN CASE. UPDATE: WE NOTICE THAT DOING THIS DOES SPEED UP THE
# PROPERTY SEARCHES INTERESTINGLY ENOUGH.
DATASET_DTYPES = {'closed_roll_year': 'Int64', 'property_location': 'object', 'parcel_number': 'object', 'block': 'object',
'lot': 'object', 'volume_number': 'float16', 'use_code': 'object', 'use_definition': 'object', 'property_class_code': 'object',
'property_class_code_definition': 'object', 'year_property_built': 'float16', 'number_of_bathrooms': 'float16', 
'number_of_bedrooms': 'float16', 'number_of_rooms': 'float16', 'number_of_stories': 'float16', 'number_of_units': 'float16',
'zoning_code': 'object', 'construction_type': 'object', 'lot_depth': 'float16', 'lot_frontage': 'float16', 'property_area': 'float32',
'basement_area': 'float32', 'lot_area': 'float32', 'lot_code': 'object', 'tax_rate_area_code': 'float16', 'percent_of_ownership': 'float32',
'exemption_code': 'object', 'exemption_code_definition': 'object', 'status_code': 'object', 'misc_exemption_value': 'float32',
'homeowner_exemption_value': 'float32', 'current_sales_date': 'object', 'assessed_fixtures_value': 'float32', 
'assessed_improvement_value': 'Int64', 'assessed_land_value': 'Int64', 'assessed_personal_property_value': 'Int64',
'assessor_neighborhood_district': 'category', 'assessor_neighborhood_code': 'category', 'assessor_neighborhood': 'category',
'supervisor_district': 'category', 'analysis_neighborhood': 'category', 'the_geom': 'object', 'row_id': 'object'}

# FLAGS
functionSelected = ''
propertyListImported = False
propertyListRunning = False
propertyDataSetImported = False
propertySearchExecuted = False

# GLOBAL VARIABLES
# The Pandas DF of the tax data to be used throughout the program. Temporarily declare as a string.
taxDataSet = ''
# The Pandas DF for property searching. Temporarily declare as a string.
propertySearchData = ''
# The Pandas DF for properties found by searching. Temporarily declare as string.
propertyFoundData = ''
# The Pandas DF for properties used in graphing. Temporarily declared as string.
graphDataFrame = ''
# FUNCTIONS

def display_menu():
    # TODO Pretty print the menu with justifications.
    print("\nPlease select a menu item from below:")
    print("Quit Program: Q | Ad-hoc Property Search: 1 | Import Property List: 2 | Import Data Set: 3")
    print("Run Property Search: 4 | Generate Report(s): 5 | Show Summary Report Only: 6")
    print("Generate Plots: 7")

def validate_menu_input(menuInput):
    if (menuInput.upper() == 'Q'):
        return menuInput
    if (menuInput.isdigit()):
        if (int(menuInput) in MENU_RANGE):
            return int(menuInput)
        else:
            print("-- Invalid menu item chosen, please enter again: --\n")
            display_menu()
            return False
    else:
        print("-- Invalid menu item chosen, please enter again: --\n")
        display_menu()
        return False

# Function that outputs Please wait, then periods to show program is still running (positive feedback).
def please_wait():
    print('Please wait, this may take several seconds...\n\n')

def function_select(menuItemChosen):
    functionError = ''
    if menuItemChosen == ADHOC_PROPERTY_SEARCH:
        functionError = do_adhoc_property_search()
    elif menuItemChosen == IMPORT_PROPERTY_LIST:
        functionError = do_import_property_list()
    elif menuItemChosen == IMPORT_DATA_SET:
        functionError = do_import_data_set()
    elif menuItemChosen == RUN_PROPERTY_SEARCH:
        functionError = do_run_property_search()
    elif menuItemChosen == GENERATE_REPORT:
        functionError = do_generate_report()
    elif menuItemChosen == SUMMARY_REPORT_ONLY:
        functionError = do_summary_report()
    elif menuItemChosen == CREATE_PLOTS:
        functionError = do_make_plots()
    else:
        exit('Oops. A function type did not match somehow.')
    
    return functionError

def do_adhoc_property_search():
    global propertyDataSetImported
    anError = ''
    if not propertyDataSetImported:
        return '\n-- Please import the property data set first. Menu item ' + str(IMPORT_DATA_SET) + '--'
    
    # Initialize while loop
    searchType = 0
    # Make sure input is between 1 and 2.
    while searchType not in range(1,3):
        searchType = input('Please enter search type. 1 for address search. 2 for parcel number search: ')
        # Cast search type to int for range comparison.
        try:
            searchType = int(searchType)
        except TypeError:
            anError = 'Invalid input. Please enter again\n'
            return anError
        
    if (searchType == ADDRESS_SEARCH):
        # Perform ad-hoc, address search type
        property_search(ADHOC_SEARCH, ADDRESS_SEARCH)
        
    if (searchType == PARCEL_SEARCH):
        # Perform ad-hoc, parcel number search type
        property_search(ADHOC_SEARCH, PARCEL_SEARCH)
    return anError


def do_import_property_list():
    # Use the property list imported flag
    global propertyListImported
    # Use the propertySearchData global variable
    global propertySearchData
    # Set function errors to empty string.
    anError = ''

    print('Please enter the path and filename for the property list (in CSV format):')
    # Get the path and file name for the property list file
    openPropertyList = input()
    # Print please wait prompt.
    please_wait()
    # Try except block to try and open the property list file
    try:
        propertyList = pd.read_csv(openPropertyList)
    # Catch exception file not found and display the error
    except FileNotFoundError:
        anError = "Unable to open property list: File not found."
        return anError
    # Catch exception path given is a directory (no file name specified) and display the error
    except IsADirectoryError:
        anError = "Unable to open property list: Path specified is a directory."
        return anError
    # Catch all, something else went wrong, display unknown error with exception error.
    except Exception as e:
        anError = "Unable to open property list, unknown error: " + repr(e)
        return anError
    else:
        print("Property list imported. Checking for minimum required data...")

    # Checking minimum (correct) data headers exist. Create a set from the mimimum data global constant.
    # Compare that to see if it is a subset of the headers that exist in the data file.
    if (not(set(PROPERTY_LIST_MINIMUM_DATA).issubset(propertyList.columns.values.tolist()))):
        anError = "Imported property list does not meet the minimum requirements"
        return anError
    # For debugging
    #print(propertyList.head())
    print("Property list has been successfully imported.")
    # Assign local property list varible to global variable
    propertySearchData = propertyList
    # Set global flag that property list import has been completed.
    propertyListImported = True
    # Return empty string.
    return anError

def do_import_data_set():
    # Use the data set imported flag
    global propertyDataSetImported
    # Use the taxDataSet global variable
    global taxDataSet
    # Set function errors to empty string.
    anError = ''

    print('Please enter the path and filename for the property tax dataset (in CSV format):')
    # Get the path and file name for the tax data set
    openDataSet = input()
    # Print please wait prompt.
    please_wait()
    # Try except block to try and open the tax dataset file
    try:
        taxDataSet = pd.read_csv(openDataSet, dtype=DATASET_DTYPES)
    # Catch exception file not found and display the error
    except FileNotFoundError:
        anError = "Unable to open tax data set: File not found."
        return anError
    # Catch exception path given is a directory (no file name specified) and display the error
    except IsADirectoryError:
        anError = "Unable to open tax data set: Path specified is a directory."
        return anError
    # Catch all, something else went wrong, display unknown error with exception error.
    except Exception as e:
        anError = "Unable to open tax data set, unknown error: " + repr(e)
        return anError
    else:
        print("Property tax dataset imported. Checking for minimum required data...")
    
    # For debugging
    #print (set(TAX_DATASET_MINIMUM_DATA).issubset(taxDataSet.columns.values.tolist()))
    
    # Checking minimum (correct) data headers exist. Create a set from the mimimum data global.
    # Compare that to see if it is a subset of the headers that exist in the data file.
    if (not(set(TAX_DATASET_MINIMUM_DATA).issubset(taxDataSet.columns.values.tolist()))):
        anError = "Imported dataset does not meet the minimum requirements"
        return anError
    # For debugging
    #print(taxDataSet.head())

    print("Tax dataset has been successfully imported.")
    # Set global flag that property import has been completed.
    propertyDataSetImported = True
    # Return empty string.
    return anError

# Reusable search function to prevent having several search functions implemented in other functions.
# singleOrMulti determines ad hoc vs multiple property search, address or parcel determines what to search by.
def property_search(singleOrMulti, addressOrParcel):
    global propertySearchExecuted
    global propertyFoundData
    anError=''
    # If an ad-hoc search is specified
    if (singleOrMulti == ADHOC_SEARCH):
        # If an address search is specified
        if (addressOrParcel == ADDRESS_SEARCH):
            # Set which dataframe column to search on
            searchField = 'property_location'
            # Prompt for input on the address to search
            searchExpression = input('Enter an address to search. Do NOT input the street type (st, ave, etc.): ').upper()
        #If a parcel number search is specified
        if (addressOrParcel == PARCEL_SEARCH):
            # Set which dataframe column to search on
            searchField = 'parcel_number'
            # Prompt for input on the parcel number to search
            searchExpression = input('Enter a parcel number to search. Note: Parcel numbers are not validated: ').upper()
        # If there are any results for the search term inputted    
        if (taxDataSet[searchField].str.contains(searchExpression).any()):
            # Property was found in dataset
            print('\nProperty found!')
            # Prompt for a tax year
            print('Please enter a tax year: ')
            # Try to input a tax year
            try:
                yearSearch = int(input())
            # If there is an TypeError exception then the user probably did not enter an integer
            except TypeError:
                anError = "Please enter a four digit numerical year"
                return anError
            # Assign all found rows of tax year entered to foundProperty DF
            foundProperties = taxDataSet.loc[taxDataSet[searchField].str.contains(searchExpression)]
            foundProperties = foundProperties.loc[taxDataSet['closed_roll_year'] == yearSearch]
            # If the DF is empty then no tax data was found for the year inputted.
            if foundProperties.empty:
                anError = "No tax data found for the year entered!"
                return anError
            else:
                # Reset indices to zero (for iloc)
                foundProperties = foundProperties.reset_index()

        else:
            anError = 'Property not found. Please try again.'
            return anError
    #If the user selects search by property list
    if (singleOrMulti == MULTI_SEARCH):
        # If propertySearchData is an empty string, the user has not run the property list import. Output error
        if (not propertyListImported):
            anError = "Please import property list import first. Menu item " + str(IMPORT_PROPERTY_LIST)
            return anError
        # If propertySearchData DF is empty, then the CSV data file was likely empty. Output error.
        if (propertySearchData.empty):
            anError = "Property search list was empty please re-run with a new file. Menu item " + str(IMPORT_PROPERTY_LIST)
            return anError
        
        # Check to see if property search file contains addresses or parcel numbers
        # If all of the columns in property location are NA, use parcel numbers
        if (propertySearchData['property_location'].isna().all()):
            searchType = 'parcel_number'
        elif (propertySearchData['parcel_number'].isna().all()):
            searchType = 'property_location'
        else:
            # Both columns are empty, return an error
            anError = 'Both of the property_search and parcel_number columns are empty. Please try again with a new file'
            return anError
        # Find properties from the tax dataset that are in the property search dataset using the isin method
        please_wait()
        foundProperties = taxDataSet[taxDataSet[searchType].isin(propertySearchData[searchType])]

        # Next 2 lines For troubleshooting
        #print(searchType)
        #print(foundProperties)

        # If I have time add support for search by address and parcel number
        # Reformat addresses, not sure how to do this any other way than a loop.
        for i in range(0,len(foundProperties.index)):
            #print(foundProperties.iloc[i]['property_location'])
            addressReplace = re.match(ADDRESS_REGEX, foundProperties.iloc[i]['property_location'])
            # Replace all property_location (address) with regex formatted address to make it more readable.
            # Sometimes addresses don't match the typical address format, skip these
            if addressReplace is not None:
                # Also strip leading zeroes in the address which is in the data set
                # Had to do it this way because I was getting view copy assignment errors
                foundProperties.iloc[i, foundProperties.columns.get_loc('property_location')] = addressReplace.group(1).lstrip('0') + addressReplace.group(2)
            
            # For troubleshooting
            #print(foundProperties.iloc[i]['property_location'])

    # Formatter to put tas values into a currency format
    currencyFormat = '${:,}'
    # Sum totals of all listings of property found for the tax year. Four different tax assessments. This is a basic summary.
    totalFixturesValue = foundProperties['assessed_fixtures_value'].sum()
    totalImprovementValue = foundProperties['assessed_improvement_value'].sum()
    totalLandValue = foundProperties['assessed_land_value'].sum()
    totalPersonalPropertyValue = foundProperties['assessed_personal_property_value'].sum()
    # Calculate total taxable value
    totalTaxableValue = totalFixturesValue + totalImprovementValue + totalLandValue + totalPersonalPropertyValue

    # Assign total taxable value to a new column in the foundProperties DF
    foundProperties['total_tax_value'] = foundProperties['assessed_fixtures_value'] + foundProperties['assessed_improvement_value'] + foundProperties['assessed_land_value'] + foundProperties['assessed_personal_property_value']
    # Reset indexes
    foundProperties.reset_index(inplace=True, drop=True)

    #For troubleshooting
    #print(foundProperties)
    
    # Using the property tax yearly rates dictionary, calculate the tax liability (cost) for every entry mulitplying total tax value by the tax rate for that year
    foundProperties['total_tax_liability'] = foundProperties['total_tax_value'].mul(foundProperties['closed_roll_year'].map(PROPERTY_TAX_YEARLY_RATES))
    # Round the new total tax liability column to 2 decimal places
    foundProperties.total_tax_liability = foundProperties.total_tax_liability.round(2)
    # Get sum of tax liabilities
    totalTaxLiability = foundProperties['total_tax_liability'].sum()

    # Print out summaries of taxable values and number of properties
    print('Total properties found in dataset: ' + str(len(foundProperties.index)))
    print('Total Fixtures Taxable Value: ' + currencyFormat.format(totalFixturesValue).rjust(11, ' '))
    print('Total Improvements Taxable Value: ' + currencyFormat.format(totalImprovementValue).rjust(15, ' '))
    print('Total Land Taxable Value: ' + currencyFormat.format(totalLandValue).rjust(23, ' '))
    print('Total Personal Property Taxable Value: ' + currencyFormat.format(totalPersonalPropertyValue).rjust(2, ' '))
    print('Total Taxable Value: ' + currencyFormat.format(totalTaxableValue).rjust(28, ' '))
    print('Total Tax Liability: ' + currencyFormat.format(totalTaxLiability).rjust(28, ' '))
    
    # For troubleshooting
    foundProperties.to_csv('PropertySearchResults.csv', index=False)

    # Assign the local property dataframe to the global one for other functions to use
    propertyFoundData = foundProperties
    # Mark the flag that the property searh has been executed to true
    propertySearchExecuted = True
    # Return any errors
    return anError

def do_run_property_search():
    anError = ''
    # Make sure the prerequisite menu functions have been performed first
    if not propertyDataSetImported:
        return '\n-- Please import the property data set first. Menu item ' + str(IMPORT_DATA_SET) + '--'
    if not propertyListImported:
        return '\n-- Please import the property search list first. Menu item ' + str(IMPORT_PROPERTY_LIST) + '--'
    
    # CURRENTLY PROPERTY DATA FILE SEARCH SUPPORTS ONLY ALL ADDRESS OR ALL PARCEL NUMBER SEARCH

    print('Currently, only data files that are all addresses or all parcel numbers (not a mix) are supported')

    anError = property_search(MULTI_SEARCH, ADDRESS_SEARCH)
    return anError

def do_generate_report():
    anError = ''
    # Make sure the prerequisite menu functions have been performed first
    if (not propertySearchExecuted):
        anError = 'Please execute property search before creating plots. Note: Plots are not available for ad-hoc searches'

    # Set up input validation loop
    else:
        menuSelection = 0
        
        # While the input is invalid
        while menuSelection not in range(1,9):

        ### OUTPUT REPORT MENU
            print("Please choose a report option:\n")
            print("Generate all reports: 1".ljust(30, ' '), "Property Values Only: 2".center(30, ' '), "Property Tax Owed Only: 3".rjust(30, ' '))
            print("Property Values and Tax Owed: 4".ljust(30, ' '), "Total Tax By Property Type: 5".center(30, ' '), "Total Tax Per Neighborhood: 6".rjust(30, ' '))

            # Input menu selection from user
            menuSelection = int(input())
            # Print the menu item chosen is not in range
            if menuSelection not in range (1,9):
                print('Invalid menu selection made, please choose 1 through 8:\n')
        
        # If option 1 is chosen output all reports
        if menuSelection==1:
            # loop through all reports
            for i in range(2,9):
                make_report(i)
            return anError

        # Else run the indiviual report requested
        else:
            make_report(menuSelection)
        return anError

# Required so we can loop through the reports if all reports are selected
def make_report(menuSelection):
    anError = ''

    # Get a UNIX style timestamp to append to the filename so that multiple report runs have unique file names
    timestamp = int(time.time())

    # Output property list with taxable values only
    if menuSelection==2:
        # Select all rows with the appropriate columns for the report, output the CSV file with timestamp, let user know.
        reportDataFrame = propertyFoundData.loc[:, ['closed_roll_year', 'property_location', 'parcel_number', 'total_tax_value']]
        reportFileName = 'PropertyValuesOnly-' + str(timestamp) + '.csv'
        reportDataFrame.to_csv(reportFileName, index=False)
        print('Report generated and saved to: ', reportFileName)

    # Output property list with tax liabilities (cost) only
    if menuSelection==3:
        # Select all rows with the appropriate columns for the report, output the CSV file with timestamp, let user know.
        reportDataFrame = propertyFoundData.loc[:, ['closed_roll_year', 'property_location', 'parcel_number', 'total_tax_liability']]
        reportFileName = 'PropertyTaxOnly-' + str(timestamp) + '.csv'
        reportDataFrame.to_csv(reportFileName, index=False)
        print('Report generated and saved to: ', reportFileName)

    # Output property list with taxable values and tax liabilities (cost)
    if menuSelection==4:
        # Select all rows with the appropriate columns for the report, output the CSV file with timestamp, let user know.
        reportDataFrame = propertyFoundData.loc[:, ['closed_roll_year', 'property_location', 'parcel_number', 'total_tax_value', 'total_tax_liability']]
        reportFileName = 'PropertyValuesandTax-' + str(timestamp) + '.csv'
        reportDataFrame.to_csv(reportFileName, index=False)
        print('Report generated and saved to: ', reportFileName)

    if menuSelection==5:
        # Aggregate property type and output with the appropriate columns for the report, output the CSV file with timestamp, let user know.
        reportDataFrame = propertyFoundData.groupby('property_class_code_definition')['total_tax_liability'].sum().reset_index(name='total_tax_liability')
        # For troubleshooting
        #print(reportDataFrame)
        reportFileName = 'PropertyTaxLiabilityByPropertyType-' + str(timestamp) + '.csv'
        reportDataFrame.to_csv(reportFileName, index=False)      
        print('Report generated and saved to: ', reportFileName)
        return anError

    if menuSelection==6:
        # Aggregate neighborhood and output with the appropriate columns for the report, output the CSV file with timestamp, let user know.
        #anError = 'Not yet implemented.'
        reportDataFrame = propertyFoundData.groupby('analysis_neighborhood')['total_tax_liability'].sum().reset_index(name='total_tax_liability')
        # For troubleshooting
        #print(reportDataFrame)
        reportFileName = 'PropertyTaxLiabilityByNeighborhood-' + str(timestamp) + '.csv'
        reportDataFrame.to_csv(reportFileName, index=False)
        print('Report generated and saved to: ', reportFileName)
        return anError

    if menuSelection==7:
        # Aggregate property type and output with the appropriate columns for the report, output the CSV file with timestamp, let user know.
        #anError = 'Not yet implemented.'
        reportDataFrame = propertyFoundData.groupby('property_class_code_definition')['total_tax_value'].sum().reset_index(name='total_tax_value')
        # For troubleshooting
        #print(reportDataFrame)
        reportFileName = 'PropertyTaxValueByPropertyType-' + str(timestamp) + '.csv'
        reportDataFrame.to_csv(reportFileName, index=False)
        print('Report generated and saved to: ', reportFileName)
        return anError

    if menuSelection==8:
        # Aggregate neighborhood and output with the appropriate columns for the report, output the CSV file with timestamp, let user know.
        #anError = 'Not yet implemented.'
        reportDataFrame = propertyFoundData.groupby('analysis_neighborhood')['total_tax_value'].sum().reset_index(name='total_tax_value')
        # For troubleshooting
        #print(reportDataFrame)
        reportFileName = 'PropertyTaxValueByNeighborhood-' + str(timestamp) + '.csv'
        reportDataFrame.to_csv(reportFileName, index=False)
        print('Report generated and saved to: ', reportFileName)        
        return anError

    return anError


def do_summary_report():
    anError = ''

    # Calculate tax liability

    ## Calculate Sums ##
    # Sum totals of all listings of property found for the tax year. Four different tax assessments. This is a basic summary.
    totalFixturesValue = propertyFoundData['assessed_fixtures_value'].sum()
    totalImprovementsValue = propertyFoundData['assessed_improvement_value'].sum()
    totalLandValue = propertyFoundData['assessed_land_value'].sum()
    totalPersonalPropertyValue = propertyFoundData['assessed_personal_property_value'].sum()
    # Calculate total taxable value
    totalTaxableValue = totalFixturesValue + totalImprovementsValue + totalLandValue + totalPersonalPropertyValue
    #print(propertyFoundData)

    # Get sum of tax liabilities
    totalTaxLiability = propertyFoundData['total_tax_liability'].sum()

    # For troubleshooting
    #print(propertyFoundData)
    #propertyFoundData.to_csv('totaltaxtest.csv', index=False)
    ## Calculate Means ##
    fixturesMeanValue = propertyFoundData['assessed_fixtures_value'].mean()
    improvementsMeanValue = propertyFoundData['assessed_improvement_value'].mean()
    landMeanValue = propertyFoundData['assessed_land_value'].mean()
    personalMeanValue = propertyFoundData['assessed_personal_property_value'].mean()
    totalMeanValue = propertyFoundData['total_tax_value'].mean()
    totalMeanLiability = propertyFoundData['total_tax_liability'].mean()

    # Calculate standard deviations for taxable values
    fixturesSTDValue = propertyFoundData['assessed_fixtures_value'].std()
    improvementsSTDValue = propertyFoundData['assessed_improvement_value'].std()
    landSTDValue = propertyFoundData['assessed_land_value'].std()
    personalSTDValue = propertyFoundData['assessed_personal_property_value'].std()
    totalSTDValue = propertyFoundData['total_tax_value'].std()
    totalSTDLiability = propertyFoundData['total_tax_liability'].std()
    
    # Calculate max and min value and liabilities
    maxTaxableValue = propertyFoundData['total_tax_value'].max()
    minTaxableValue = propertyFoundData['total_tax_value'].min()
    maxTaxLiability = propertyFoundData['total_tax_liability'].max()
    minTaxLiability = propertyFoundData['total_tax_liability'].min()
    ## OUTPUT BLOCK ##
    # Format currencies with commas and 2 decimal places
    currencyFormat = '${:,.2f}'
    print("\nTOTALS:")
    print("Sum of fixtures taxable value: ".ljust(41, ' '), currencyFormat.format(totalFixturesValue))
    print("Sum of improvements taxable value: ".ljust(41, ' '), currencyFormat.format(totalImprovementsValue))
    print("Sum of land taxable value: ".ljust(41, ' '), currencyFormat.format(totalLandValue))
    print("Sum of personal property taxable value: ".ljust(41, ' '), currencyFormat.format(totalPersonalPropertyValue))
    print("TOTAL TAXABLE VALUE:".ljust(41, ' '), currencyFormat.format(totalTaxableValue))
    print("TOTAL TAX LAIBILITY:".ljust(41, ' '), currencyFormat.format(totalTaxLiability))

    print("\nMEANS:")
    print("Mean of fixtures taxable value: ".ljust(41, ' '), currencyFormat.format(fixturesMeanValue))
    print("Mean of improvements taxable value: ".ljust(41, ' '), currencyFormat.format(improvementsMeanValue))
    print("Mean of land taxable value: ".ljust(41, ' '), currencyFormat.format(landMeanValue))
    print("Mean of personal property taxable value: ".ljust(41, ' '), currencyFormat.format(personalMeanValue))
    print("MEAN OF TOTAL TAXABLE VALUE:".ljust(41, ' '), currencyFormat.format(totalMeanValue))
    print("MEAN OF TOTAL TAX LAIBILITY:".ljust(41, ' '), currencyFormat.format(totalMeanLiability))

    print("\nSTANDARD DEVIATIONS:")
    print("STD of fixtures taxable value: ".ljust(41, ' '), currencyFormat.format(fixturesSTDValue))
    print("STD of improvements taxable value: ".ljust(41, ' '), currencyFormat.format(improvementsSTDValue))
    print("STD of land taxable value: ".ljust(41, ' '), currencyFormat.format(landSTDValue))
    print("STD of personal property taxable value: ".ljust(41, ' '), currencyFormat.format(personalSTDValue))
    print("STD OF TOTAL TAXABLE VALUE:".ljust(41, ' '), currencyFormat.format(totalSTDValue))
    print("STD OF TOTAL TAX LAIBILITY:".ljust(41, ' '), currencyFormat.format(totalSTDLiability))

    print('\nRANGES:')
    print("Max taxable property value:".ljust(41, ' '), currencyFormat.format(maxTaxableValue))
    print("Min taxable property value:".ljust(41, ' '), currencyFormat.format(minTaxableValue))
    print("Max tax liability:".ljust(41, ' '), currencyFormat.format(maxTaxLiability))
    print("Min tax liability:".ljust(41, ' '), currencyFormat.format(minTaxLiability))

    anError =''
    return anError


def do_make_plots():
    global graphDataFrame
    # Make sure the prerequisite menu items have been performed.
    if (not propertySearchExecuted):
        anError = 'Please execute property search before creating plots. Note: Plots are not available for ad-hoc searches'

    else:
        ###### VISUALIZATION STUFF GOES HERE ######
        anError = ''
        graphDataFrame = propertyFoundData.loc[:, ['total_tax_value','total_tax_liability','analysis_neighborhood','property_class_code_definition']]
        # tuples used for input validation
        plotSelection = 0
        print("Please choose a pie chart option:\n")
        print("Generate all pie charts: 1 |".ljust(30, ' '), " Aggregate by Neighborhood Only: 2 |".center(30, ' '), " Aggregate by Building Use Code Only: 3 |".rjust(30, ' '))
        print('Histogram only: 4 |')
        plotSelection = int(input())
        if plotSelection not in range(1,5):
            plotSelection = int(input('Invalid input. Please enter pie chart option 1-3:'))
        if plotSelection == 1:
            for i in range(2,5):
                make_pie_chart(i)
        else:
            make_pie_chart(plotSelection)
            

        
        
    return anError

def make_pie_chart(plotSelection):
    savePlots = ''
    validateList = ('y','Y','Yes','YES','yes','n','N','No','NO','no')
    yesList = ('y','Y','Yes','YES','yes')
    while savePlots not in validateList:
        savePlots = input('Would you like to save plots?')
        if savePlots not in validateList:
            print('Invalid input.')
    if plotSelection == 2:
        neighborhoodValueDataFrame = graphDataFrame.groupby('analysis_neighborhood')['total_tax_value'].sum().reset_index(name='total_tax_value')
        neighborhoodValueDataFrame = neighborhoodValueDataFrame[neighborhoodValueDataFrame['total_tax_value'] != 0]
        plt.figure(1)
        plt.pie(neighborhoodValueDataFrame['total_tax_value'],labels=neighborhoodValueDataFrame['analysis_neighborhood'])
        plt.title('Total Tax Value of Neighborhoods')
        if savePlots in yesList:
            plt.savefig('NeighborhoodTaxableValue.png')
        plt.show()
        neighborhoodLiabilityDataFrame = graphDataFrame.groupby('analysis_neighborhood')['total_tax_liability'].sum().reset_index(name='total_tax_liability')
        neighborhoodLiabilityDataFrame = neighborhoodLiabilityDataFrame[neighborhoodLiabilityDataFrame['total_tax_liability'] != 0]
        plt.figure(2)
        plt.pie(neighborhoodLiabilityDataFrame['total_tax_liability'],labels=neighborhoodValueDataFrame['analysis_neighborhood'])
        plt.title('Total Tax Liability of Neighborhoods')
        if savePlots in yesList:
            plt.savefig('NeighborhoodTaxLiability.png')
        plt.show()
    if plotSelection == 3:
        useCodeValueDataFrame = graphDataFrame.groupby('property_class_code_definition')['total_tax_value'].sum().reset_index(name='total_tax_value')
        useCodeValueDataFrame = useCodeValueDataFrame[useCodeValueDataFrame['total_tax_value'] != 0]
        plt.figure(3)
        plt.pie(useCodeValueDataFrame['total_tax_value'],labels=useCodeValueDataFrame['property_class_code_definition'])
        plt.title('Total Tax Value of Use Codes')
        if savePlots in yesList:
            plt.savefig('UseCodeTaxableValue.png')
        useCodeLiabilityDataFrame = graphDataFrame.groupby('property_class_code_definition')['total_tax_liability'].sum().reset_index(name='total_tax_liability')
        useCodeLiabilityDataFrame = useCodeLiabilityDataFrame[useCodeLiabilityDataFrame['total_tax_liability'] != 0]
        plt.figure(4)
        plt.pie(useCodeLiabilityDataFrame['total_tax_liability'],labels=useCodeLiabilityDataFrame['property_class_code_definition'])
        plt.title('Total Tax Liability of Use Codes')
        if savePlots in yesList:
            plt.savefig('UseCodeTaxLiability.png')
    if plotSelection == 4:
        plt.figure(5)
        plt.hist(graphDataFrame['total_tax_value'],bins=10)
        plt.title('Property Values')
        plt.xlabel("Value Ranges")
        plt.ylabel("Frequency")
        plt.savefig("PropertyValueHistogram.png")


# MAIN MENU LOOP
# Print welcome line
print("Welcome to the property tax evaluation program..")

##### MAIN LOOP #####
# Loop until the program is forced to exit.
while True:

    # Display the menu, and validate menu entry until a valid entry is made
    display_menu()
    menuItem = input()
    validate = validate_menu_input(menuItem)
    while validate == False:
        menuItem = input()
        validate = validate_menu_input(menuItem)

    # Quit program with validation
    if (menuItem.upper() == 'Q'):
        print('-- ARE YOU SURE YOU WANT TO QUIT? (y/N) --')
        quitYesNo = input()
        if (quitYesNo.upper() == 'Y'):
            # End program
            quit()

    # Menu input validated, output selected item and perform function requested.
    print('You have selected menu item: ' + str(menuItem) +'\n')
    
    # Perform the function requested, function_select returns an string, an error description or blank string.
    functionError = function_select(int(menuItem))

    # If the error string is not empty string, display the error, return to the main menu.
    if functionError:
        print(functionError+'\n\n')


Welcome to the property tax evaluation program..

Please select a menu item from below:
Quit Program: Q | Ad-hoc Property Search: 1 | Import Property List: 2 | Import Data Set: 3
Run Property Search: 4 | Generate Report(s): 5 | Show Summary Report Only: 6
Generate Plots: 7
1
You have selected menu item: 1


-- Please import the property data set first. Menu item 3--



Please select a menu item from below:
Quit Program: Q | Ad-hoc Property Search: 1 | Import Property List: 2 | Import Data Set: 3
Run Property Search: 4 | Generate Report(s): 5 | Show Summary Report Only: 6
Generate Plots: 7
